In [1]:
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns 
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer # TF-IDF
from sklearn.feature_extraction.text import CountVectorizer # Bag of word
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier 
from sklearn.naive_bayes import MultinomialNB 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from pyvi import ViTokenizer, ViPosTagger

# You can also import other things ...
# YOUR CODE HERE (OPTION)
import re
from sklearn import preprocessing

In [2]:
import sklearn
sklearn.__version__

'0.21.3'

In [3]:
df1 = pd.read_csv("data.txt", sep = "\t", encoding = "utf-8", header = None)
df1.columns = ['Link', 'Topic','Title', 'Content']

In [4]:
df1.describe()

,Link,Topic,Title,Content
count,151487,151487,151487,140813
unique,145258,5,137060,124129
top,https://vnexpress.net/khoa-hoc/linh-duong-non-...,Thế giới,Năm câu đố tìm điểm bất hợp lý,Mời độc giả đặt câu hỏi tại đây.
freq,140,32806,141,595


In [5]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151487 entries, 0 to 151486
Data columns (total 4 columns):
Link       151487 non-null object
Topic      151487 non-null object
Title      151487 non-null object
Content    140813 non-null object
dtypes: object(4)
memory usage: 4.6+ MB


In [6]:
df1.head()

,Link,Topic,Title,Content
0,https://vnexpress.net/giao-duc/bon-cau-do-do-k...,Giáo dục,Bốn câu đố đo khả năng phản xạ,Câu 1: Một cậu bé 7 tuổi đứng từ cửa sổ tầng 6...
1,https://vnexpress.net/giao-duc/thay-giao-singa...,Giáo dục,Thầy giáo Singapore đi tù vì ấu dâm học sinh,"Ngày 22/4, tòa án ở Singapore tuyên phạt một g..."
2,https://vnexpress.net/giao-duc/de-xuat-giao-du...,Giáo dục,Đề xuất giáo dục phổ thông trong 11 năm,Tại hội nghị phản biện Luật Giáo dục (sửa đổi)...
3,https://vnexpress.net/giao-duc/phu-huynh-my-ke...,Giáo dục,Phụ huynh Mỹ kêu gọi sa thải giáo viên vì phê ...,"Chris Piland, sống ở bang Pennsylvania, Mỹ phẫ..."
4,https://vnexpress.net/giao-duc/ngoi-truong-dan...,Giáo dục,Ngôi trường dành riêng cho học sinh bị bắt nạt...,Nạn bắt nạt gây ảnh hưởng tiêu cực đến cuộc số...


**_BƯỚC 01: TIỀN XỬ LÍ DỮ LIỆU_**

In [7]:
df = df1.copy()

df.drop(df[df['Content'].str.len() <= 50].index, inplace = True) # xóa các dòng có len(Content) <= 50 
df = df.drop_duplicates()
df = df.dropna()
df['X'] = df['Title'] + " " + df['Content']
df['X'].astype(str)
df = df.drop(["Link", "Title", "Content"], axis = 1)
df.head()

,Topic,X
0,Giáo dục,Bốn câu đố đo khả năng phản xạ Câu 1: Một cậu ...
1,Giáo dục,Thầy giáo Singapore đi tù vì ấu dâm học sinh N...
2,Giáo dục,Đề xuất giáo dục phổ thông trong 11 năm Tại hộ...
3,Giáo dục,Phụ huynh Mỹ kêu gọi sa thải giáo viên vì phê ...
4,Giáo dục,Ngôi trường dành riêng cho học sinh bị bắt nạt...


In [8]:
df.describe(include = "all")

,Topic,X
count,129106,129106
unique,5,122662
top,Pháp luật,"Bí quyết cải thiện 2,5 điểm IELTS sau 380 giờ ..."
freq,31515,3


In [9]:
# Tổng các dòng thiếu giá trị 
print(df['X'].isnull().sum().sum())
df.head()

0


,Topic,X
0,Giáo dục,Bốn câu đố đo khả năng phản xạ Câu 1: Một cậu ...
1,Giáo dục,Thầy giáo Singapore đi tù vì ấu dâm học sinh N...
2,Giáo dục,Đề xuất giáo dục phổ thông trong 11 năm Tại hộ...
3,Giáo dục,Phụ huynh Mỹ kêu gọi sa thải giáo viên vì phê ...
4,Giáo dục,Ngôi trường dành riêng cho học sinh bị bắt nạt...


In [10]:
# Tổng số bài báo lấy được từ data thô
print(len(df))

# Số lượng các thể loại bài báo 
s = df.groupby(['Topic']).count()
s

129106


,X
Topic,
Giáo dục,21543
Khoa học,19940
Pháp luật,31515
Thế giới,29987
Thời sự,26121


In [11]:
# Tách thành 1 series là y và dataFrame X
y_sr = df["Topic"] 
X_df = df.drop("Topic", axis=1)

In [12]:
# Tách tập train, tập validation, tập test theo tỉ lệ 6:2:2
train_X_df, test_X_df, train_y_sr, test_y_sr = train_test_split(X_df, y_sr, test_size=0.2, stratify = y_sr, random_state=0)

In [13]:
# Tách tập train, tập validation, tập test theo tỉ lệ 6:2:2
X_train, val_X_df, y_train, val_y_sr = train_test_split(train_X_df,train_y_sr, stratify = train_y_sr, random_state=0)

In [14]:
#LƯU TRAIN, TEST, VAL RA 3 FILE CSV

#train = X_train
#train['y'] = y_train

#val = val_X_df
#val['y'] = val_y_sr

#test = test_X_df
#test['y'] = test_y_sr

#train.to_csv("train.csv", sep = "\t", index = None)
#val.to_csv("val.csv", sep = "\t", index = None)
#test.to_csv("test.csv", sep = "\t", index = None)

In [15]:
# XEM DỮ LIỆU TẬP TRAIN
train_X_df.describe()

,X
count,103284
unique,99075
top,Bài toán 'xác định đồng tiền giả' Hai đồng tiề...
freq,3


In [16]:
# XEM DỮ LIỆU TẬP VALIDATION
val_X_df.describe()

,X
count,25821
unique,25558
top,Nạn đập kính ôtô trộm cắp tài sản Trộm phá vỡ ...
freq,2


In [17]:
# XEM DỮ LIỆU TẬP TEST
test_X_df.describe()

,X
count,25822
unique,25573
top,Cô gái Việt tử vong trong căn hộ ở Nhật Bản Ma...
freq,3


In [18]:
# Tách từ tiếng việt
class FeatureTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.tokenizer = ViTokenizer
        self.pos_tagger = ViPosTagger
            
    def fit(self, *_):
        return self

    def transform(self, X, y=None, **fit_params):
        result = X.apply(lambda text:  self.tokenizer.tokenize(text))
        return result

**_BƯỚC 02: MÔ HÌNH HÓA DỮ LIỆU_**

***1. Naive Bayes*** 

In [19]:
Naive_Bayes_Model_pipe_line = Pipeline([("transformer", FeatureTransformer()),("vect", CountVectorizer()),("tfidf", TfidfTransformer()),("clf", MultinomialNB())])    

In [20]:
Naive_Bayes_Model_pipe_line.fit(train_X_df['X'], train_y_sr)

Pipeline(memory=None,
         steps=[('transformer', FeatureTransformer()),
                ('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=Fa

In [21]:
Naive_Bayes_Model_pipe_line.predict(val_X_df['X'])

array(['Thế giới', 'Giáo dục', 'Thế giới', ..., 'Thế giới', 'Thời sự',
       'Giáo dục'], dtype='<U9')

In [22]:
score1 = Naive_Bayes_Model_pipe_line.score(val_X_df['X'], val_y_sr)
print(score1)

0.9116997792494481


***2. Randome Forest*** 

In [23]:
RandomForest_pipe_line = Pipeline([("transformer", FeatureTransformer()),("vect", CountVectorizer()),("tfidf", TfidfTransformer()),("randomForest", RandomForestClassifier(n_jobs=5))])

In [24]:
RandomForest_pipe_line.fit(train_X_df['X'], train_y_sr)

Pipeline(memory=None,
         steps=[('transformer', FeatureTransformer()),
                ('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\...
                ('randomForest',
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features='auto',
                                        max_leaf_nodes=None,
                                        min_impur

In [25]:
RandomForest_pipe_line.predict(val_X_df['X'])

array(['Khoa học', 'Giáo dục', 'Thế giới', ..., 'Thế giới', 'Thời sự',
       'Giáo dục'], dtype=object)

In [26]:
score1 = RandomForest_pipe_line.score(val_X_df['X'], val_y_sr)
print(score1)

0.9983346888191782


**3. K-NN**

In [27]:
KNN_pipe_line = Pipeline([("transformer", FeatureTransformer()),("vect", CountVectorizer()),("tfidf", TfidfTransformer()),("KNN", KNeighborsClassifier(n_jobs=5))])

In [28]:
KNN_pipe_line.fit(train_X_df['X'], train_y_sr)

Pipeline(memory=None,
         steps=[('transformer', FeatureTransformer()),
                ('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('KNN',
                 KNeighborsClassifier(algorithm='auto', leaf_size=30,
                            

In [29]:
score1_KNN = KNN_pipe_line.score(val_X_df['X'], val_y_sr)
print(score1_KNN)

0.9343557569420239


**Qua kết quả tính độ chính xác dựa trên 3 mô hình thì nhóm quyết định chọn mô hình RandomForest**

**Tiếp theo: Chọn tham số cho mô hình RandomForest**

In [33]:
array_n_jobs = [3, 6, 7, 8, 9]
score1s = []

for i in array_n_jobs:
    RandomForest_pipe_line = Pipeline([("transformer", FeatureTransformer()),("vect", CountVectorizer()),("tfidf", TfidfTransformer()),("randomForest", RandomForestClassifier(n_jobs=i))])
    RandomForest_pipe_line.fit(train_X_df['X'], train_y_sr)
    score1 = RandomForest_pipe_line.score(val_X_df['X'], val_y_sr)
    score1s.append(score1)
    print("n_jobs = ", i)
    print("score1 = ", score1, "\n\n")

n_jobs =  3
score1 =  0.9984896014871616 


n_jobs =  6
score1 =  0.9984121451531699 


n_jobs =  7
score1 =  0.9981797761511948 


n_jobs =  8
score1 =  0.9978699508152279 


n_jobs =  9
score1 =  0.9982185043181906 




**Với các n_jobs = [3,5,6,7,8,9] thì độ chính xác của mô hình RandomForest với n_job = 3 lớn nhất nên nhóm chọn mô hình RandomForest với n_job = 3**

**_BƯỚC 03: TEST_**

In [38]:
RandomForest_pipe_line = Pipeline([("transformer", FeatureTransformer()),("vect", CountVectorizer()),("tfidf", TfidfTransformer()),("randomForest", RandomForestClassifier(n_jobs=3))])
RandomForest_pipe_line.fit(train_X_df['X'], train_y_sr)
score1_test = RandomForest_pipe_line.score(test_X_df['X'], test_y_sr)
print(score1_test)

0.8655022848733638
